# Introduction

This Notebooks is a join notebook from both the prepare_data and pytorch-bst in order to be run in google colab.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Prepare data section

In [ ]:
!pip install pytorch_lightning torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 kB 44.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 12.7 MB/s eta 0:00:00
  Created wheel for torch_geometric: filename=torch_geometric-2.3.0-py3-none-any.whl size=909897 sha256=47f0ba18c68055b1f0f3583ed7e9acc9e4d3dbf3f071f56ddb919c04519a263d
  Stored in directory: /root/.ca

In [ ]:
import pandas as pd
import torch
# import pytorch_lightning as pl
from tqdm import tqdm
import torchmetrics
import torch_geometric
import math
from urllib.request import urlretrieve
from zipfile import ZipFile
import os
import torch.nn as nn
import numpy as np
from math import sqrt
import pickle

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())

False


## Data

In [ ]:
urlretrieve('http://files.grouplens.org/datasets/movielens/ml-1m.zip', 'movielens.zip')
ZipFile('movielens.zip', 'r').extractall()

In [ ]:
users = pd.read_csv(
    "ml-1m/users.dat",
    sep="::",
    names=["user_id", "sex", "age_group", "occupation", "zip_code"],
)

ratings = pd.read_csv(
    "ml-1m/ratings.dat",
    sep="::",
    names=["user_id", "movie_id", "rating", "unix_timestamp"],
)

movies = pd.read_csv(
    "ml-1m/movies.dat", sep="::", names=["movie_id", "title", "genres"], encoding="ISO-8859-1"
)

<ipython-input-8-484fc68a9aca>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_csv(
<ipython-input-8-484fc68a9aca>:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv(
<ipython-input-8-484fc68a9aca>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv(


In [ ]:
# ratings['unix_timestamp'] = pd.to_datetime(ratings['unix_timestamp'],unit='s')
ratings['movie_id'] = ratings['movie_id'].astype(int)
ratings['user_id'] = ratings['user_id'].astype(int)
ratings = ratings.sort_values(by='unix_timestamp')
ratings

,user_id,movie_id,rating,unix_timestamp
1000138,6040,858,4,956703932
1000153,6040,2384,4,956703954
999873,6040,593,5,956703954
1000007,6040,1961,4,956703977
1000192,6040,2019,5,956703977
...,...,...,...,...
825793,4958,2399,1,1046454338
825438,4958,1407,5,1046454443
825724,4958,3264,4,1046454548
825731,4958,2634,3,1046454548


In [ ]:
# save primary csv's
if not os.path.exists('data'):
    os.makedirs('data')
    
ratings.to_csv('data/ratings.csv',index=False)

In [ ]:
users = pd.read_csv("./ml-1m/users.dat", sep="::", names = ['userID','gender','age','occupation','zipcode'])
users.head()

<ipython-input-11-84a6d861c214>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_csv("./ml-1m/users.dat", sep="::", names = ['userID','gender','age','occupation','zipcode'])


,userID,gender,age,occupation,zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [ ]:
movie_emb = pickle.load(open("movie_emb.pkl", "rb"))
genre_emb = pickle.load(open("genre_emb.pkl", "rb"))

In [ ]:
num_users = ratings['user_id'].max()
num_movies = ratings['movie_id'].max()

In [ ]:
genres = [
    "Action",
    "Adventure",
    "Animation",
    "Children's",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]
num_genres = len(genres)

genre2id = {}

for i,g in enumerate(genres):
  genre2id[g] = num_users + num_movies + i

In [ ]:
total_nodes = num_users + num_movies + num_genres
X = np.random.rand(total_nodes, 50)

for i,id in enumerate(movies['movie_id']):
  X[num_users+id, :] = movie_emb[i]

X[num_users+num_movies:, :] = genre_emb

X = torch.tensor(X, dtype=torch.float)

In [ ]:
from torch_geometric.data import Data
from torch_geometric.utils import to_dense_adj

ratings_std = ratings.copy(deep=True)
ratings_std['movie_id'] = ratings['movie_id'] + ratings['user_id'].max() - 1
ratings_std['rating'] = ratings['rating']

# User - Movie Edges
edge_list = [list(ratings_std['user_id'] - 1), list(ratings_std['movie_id'])]

rating_edges = len(edge_list[0])

# Movie - Genre Edges
for i,m in enumerate(movies['genres']):
  g = m.split('|')
  for _g in g:
    edge_list[0].append(num_users + i)
    edge_list[1].append(genre2id[_g])

genre_edges = len(edge_list[0]) - rating_edges

edge_type = list(ratings['rating'] - 1)
edge_type.extend([5]*genre_edges)

edge_index = torch.tensor(edge_list, dtype=torch.long)
edge_type = torch.tensor(edge_type, dtype=torch.long)
# adj_list = to_dense_adj(edge_index)[0]
ratings_list = torch.tensor(list(ratings_std['rating']), dtype=torch.float)

dataset = Data(x=X, y=ratings_list, edge_index=edge_index, edge_type=edge_type)

train_num = int(dataset.num_edges * 0.8)
val_num = int(dataset.num_edges * 0.1)
test_num = dataset.num_edges - train_num - val_num
dataset.train_mask = torch.cat((torch.ones(train_num, 1, dtype=torch.int), torch.zeros(val_num + test_num, 1, dtype=torch.int)))
dataset.val_mask = torch.cat((torch.zeros(train_num, 1, dtype=torch.int), torch.ones(val_num, 1, dtype=torch.int), torch.zeros(test_num, 1, dtype=torch.int)))
dataset.test_mask = torch.cat((torch.zeros(train_num + val_num, 1, dtype=torch.int), torch.ones(test_num, 1, dtype=torch.int)))

data = dataset.to(device)

print(dataset.num_node_features, dataset.num_nodes, dataset.num_edges)

50 10010 1006617


In [ ]:
import torch.nn.functional as F
from torch_geometric.nn import RGCNConv

class RGCN_NGCF(torch.nn.Module):
    def __init__(self, dataset, hidden_channels_gcn=32, hidden_channels_fc=512, dropout_probability=0.2):
        super().__init__()
        self.dataset = dataset
        self.conv1 = RGCNConv(dataset.num_node_features, hidden_channels_gcn, 6)
        self.conv2 = RGCNConv(hidden_channels_gcn, hidden_channels_gcn, 6)
        self.dropout_p = dropout_probability
        self.fc1 = nn.Linear(in_features=(2 * hidden_channels_gcn), out_features=hidden_channels_fc)
        self.fc2 = nn.Linear(in_features=hidden_channels_fc, out_features=hidden_channels_fc)
        self.output = nn.Linear(in_features=hidden_channels_fc, out_features=1)
        self.relu = nn.ReLU()

    def forward(self, data):
        x, edge_index, edge_type = data.x, data.edge_index, data.edge_type

        out = self.conv1(x, edge_index, edge_type)
        out = F.relu(out)
        out = F.dropout(out, p=self.dropout_p, training=self.training)
        out = self.conv2(out, edge_index, edge_type)
        out = F.relu(out)
        out = F.dropout(out, p=self.dropout_p, training=self.training)

        out = self.fc1(torch.cat((out[data.edge_index.T[:, 0]], out[data.edge_index.T[:, 1]]), dim=1).type(torch.float))
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.output(out)
        # out = self.relu(out)

        return out

In [ ]:
model = RGCN_NGCF(data, hidden_channels_gcn=50, hidden_channels_fc=128, dropout_probability=0.2).to(device)
print(model)

criterion = nn.MSELoss()
mae_loss = nn.L1Loss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-6)

training_rmse_vals = []
test_rmse_vals = []
val_mae_vals = []
test_mae_vals = []

num_epochs = 51
data = data.to(device)

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    out = model(data).flatten()

    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    training_rmse = loss.detach().cpu()

    model.eval()
    with torch.no_grad():
      out = model(data).flatten()
      val_rmse = criterion(out[data.val_mask], data.y[data.val_mask]).detach().cpu()
      test_rmse = criterion(out[data.test_mask], data.y[data.test_mask]).detach().cpu()
      val_mae = mae_loss(out[data.val_mask], data.y[data.val_mask]).detach().cpu()
      test_mae = mae_loss(out[data.test_mask], data.y[data.test_mask]).detach().cpu()

    training_rmse_vals.append(training_rmse)
    test_rmse_vals.append(test_rmse)
    val_mae_vals.append(val_mae)
    test_mae_vals.append(test_mae)

    # if epoch % 5 == 0:
    file_path = f"/content/drive/MyDrive/WSTM_latest/weights/kg_ngcf_latest_{epoch}.pth"
    torch.save(model.state_dict(), file_path)
    print(f'Epoch {epoch}, Training RMSE: {training_rmse}, Val RMSE: {val_rmse}')

RGCN_NGCF(
  (conv1): RGCNConv(50, 50, num_relations=6)
  (conv2): RGCNConv(50, 50, num_relations=6)
  (fc1): Linear(in_features=100, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
)
Epoch 0, Training RMSE: 11.792879104614258, Val RMSE: 21.01865577697754
Epoch 1, Training RMSE: 10.299206733703613, Val RMSE: 19.604610443115234
Epoch 2, Training RMSE: 9.325905799865723, Val RMSE: 17.97471046447754
Epoch 3, Training RMSE: 7.6942291259765625, Val RMSE: 16.056180953979492
Epoch 4, Training RMSE: 7.097343444824219, Val RMSE: 13.883712768554688
Epoch 5, Training RMSE: 6.706925868988037, Val RMSE: 11.484493255615234
Epoch 6, Training RMSE: 4.164576530456543, Val RMSE: 8.851289749145508
Epoch 7, Training RMSE: 1.9414708614349365, Val RMSE: 6.235363960266113
Epoch 8, Training RMSE: 1.7382869720458984, Val RMSE: 3.8427658081054688
Epoch 9, Training RMSE: 0.771973192691803, V

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/WSTM_latest/weights/ngcf_latest.pth")

In [ ]:
stored_model = RGCN_NGCF(data, hidden_channels_gcn=50, hidden_channels_fc=128, dropout_probability=0.2)
stored_model.load_state_dict(torch.load("/content/drive/MyDrive/WSTM_latest/weights/final_kg_weights.pth", map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
ngcf_emb = {}

In [ ]:
def get_middle_layer_output(model, input, output):
    middle_layer_output = output.detach().clone() # Get output from the last layer
    ngcf_emb['bb'] = middle_layer_output
    print(middle_layer_output)
    # return middle_layer_output

# Register the hook to the second layer
hook = stored_model.conv2.register_forward_hook(get_middle_layer_output)
out = stored_model(data)
# hook.remove()

tensor([[ 0.0114, -0.4455,  0.6853,  ...,  0.3477,  0.4807, -0.4828],
        [-0.1827, -0.5564,  0.3286,  ...,  0.4992,  0.0565, -0.6591],
        [-0.0250,  0.2214,  0.6037,  ...,  0.5270,  0.1399, -0.1662],
        ...,
        [ 0.2700,  0.1434, -0.4251,  ...,  0.2307, -0.1532,  0.8284],
        [-0.2001,  0.4299, -0.3513,  ..., -0.3569, -0.5086,  0.6812],
        [-0.0708, -0.5552, -0.6214,  ...,  0.2651, -0.1725,  0.7419]])


In [ ]:
import pickle
pickle.dump(ngcf_emb['bb'], open("kg_emb.pkl", "wb"))